In [1]:
import pandas as pd
import pymongo
import psycopg2
from sqlalchemy import create_engine
from sqlalchemy import text
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
# # create a connection to Postgres
# conn = psycopg2.connect(database="postgres", user='postgres', password='Shjais2014', host='127.0.0.1', port='5432')
# conn.autocommit = True
# cursor = conn.cursor()

# # create the database in Postgres
# create_database_sql = open('../sql/create_db.sql','r')
# cursor.execute(create_database_sql.read())

# #If no error print message
# print("Database created successfully........")

# #Closing the connection and database
# conn.close()
# create_database_sql.close()

In [3]:
csv_file = "../output_data/tweets.csv"
twitter_df = pd.read_csv(csv_file)
twitter_df.head()

,Unnamed: 0,Tweets
0,0,Every Monday I open my Sling app to watch Raw ...
1,1,happy monday y'all! | PS5 | !socials !discord ...
2,2,RT @FOX5Vegas: NLVPD Officer Alexander Cuevas ...
3,3,"RT @CrabDay: 🦀 Monday, March 29th, 2021 🦀\n\n📸..."
4,4,@TMorri_12 @WashedUpJustin_ Tony get your fell...


In [4]:
twitter_df.rename(columns = {'Tweets':'tweets'}, inplace = True) 
twitter_df.head()

,Unnamed: 0,tweets
0,0,Every Monday I open my Sling app to watch Raw ...
1,1,happy monday y'all! | PS5 | !socials !discord ...
2,2,RT @FOX5Vegas: NLVPD Officer Alexander Cuevas ...
3,3,"RT @CrabDay: 🦀 Monday, March 29th, 2021 🦀\n\n📸..."
4,4,@TMorri_12 @WashedUpJustin_ Tony get your fell...


In [5]:
# twitter_df = twitter_df.set_index("state")
# twitter_df.head()
twitter_df = twitter_df.drop(columns=['Unnamed: 0'])
twitter_df.head()

,tweets
0,Every Monday I open my Sling app to watch Raw ...
1,happy monday y'all! | PS5 | !socials !discord ...
2,RT @FOX5Vegas: NLVPD Officer Alexander Cuevas ...
3,"RT @CrabDay: 🦀 Monday, March 29th, 2021 🦀\n\n📸..."
4,@TMorri_12 @WashedUpJustin_ Tony get your fell...


In [6]:
twitter_df = twitter_df.reindex()
twitter_df.head()

,tweets
0,Every Monday I open my Sling app to watch Raw ...
1,happy monday y'all! | PS5 | !socials !discord ...
2,RT @FOX5Vegas: NLVPD Officer Alexander Cuevas ...
3,"RT @CrabDay: 🦀 Monday, March 29th, 2021 🦀\n\n📸..."
4,@TMorri_12 @WashedUpJustin_ Tony get your fell...


In [7]:
# create a connection to Postgres
conn = psycopg2.connect(database="twitter_db", user='postgres', password='Shjais2014', host='127.0.0.1', port='5432')
conn.autocommit = True
cursor = conn.cursor()

In [8]:
# run the schema for the database using the schema file created and the connection to Postgres
schema_sql = open('../sql/schema.sql','r')
cursor.execute(schema_sql.read())

#If no error print message
print("Database Schema Loaded Successfully (Tables Created)........")

# close the schema file
schema_sql.close()

Database Schema Loaded Successfully (Tables Created)........


In [9]:
#database connection
connection_string ="postgres:Shjais2014@localhost:5432/twitter_db"
engine = create_engine(f'postgresql://{connection_string}')

In [10]:
#confirm tables
engine.table_names()

['twitter_tweets']

In [11]:
### Load DataFrames into database
twitter_df.to_sql(name='twitter_tweets', con=engine, if_exists='append', index = True)

In [12]:
select_sql = open('../sql/select_df.sql','r')
twit_df = pd.read_sql_query(select_sql.read(), con=engine)

#Close File after use
select_sql.close()

In [13]:
twit_df

,tweets,index
0,Every Monday I open my Sling app to watch Raw ...,0
1,happy monday y'all! | PS5 | !socials !discord ...,1
2,RT @FOX5Vegas: NLVPD Officer Alexander Cuevas ...,2
3,"RT @CrabDay: 🦀 Monday, March 29th, 2021 🦀\n\n📸...",3
4,@TMorri_12 @WashedUpJustin_ Tony get your fell...,4
5,RT @SweetTaleBooks: Have a wonderful Monday!\n...,5
6,RT @jilevin: 911 dispatcher testifies officers...,6
7,RT @KBSWorldTV: [KPOP NEWS] BTS to Star in Spe...,7
8,RT @foosgonewild: MONDAY MOTIVATION https://t....,8
9,RT @WilliamYang120: Thousands of #Myanmar prot...,9


In [14]:
conn.close()